### Limma voom analysis of Age Dataset: RNA-seq


In [430]:
rm(list=ls())
library(data.table)
#load necessary libraries 
library(ggplot2)
library("BiocParallel")
parallelFlag=TRUE
register(MulticoreParam(50))
library(sva)
library(limma)
library(statmod)

## Load data and design

In [431]:
##load RNA-seq TPM
data=read.table('/mnt/lab_data2/annashch/nobel_lab_projects/age_V2/RNA_seq/merged_rsem/age.rna.counts.txt.tpm',header=TRUE,sep='\t')
rownames(data)=paste(data$GeneID,data$GeneName,sep="_")
data$GeneID=NULL
data$GeneName=NULL

In [432]:
head(data)

,d21_A_Rep1,d21_A_Rep2,d21_Y_Rep1,d21_Y_Rep2,d21_Y_Rep3,d0_A_Rep1,d0_A_Rep2,d0_A_Rep3,d0_A_Rep4,d0_Y_Pax7_Rep1,⋯,d5_Y_Rep2,d5_Y_Rep3,d7_A_Rep1,d7_A_Rep2,d7_A_Rep3,d7_A_Rep4,d7_Y_Rep1,d7_Y_Rep2,d7_Y_Rep3,d7_Y_Rep4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSID_gene_GeneName,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000000001_Gnai3,153.01,153.94,175.62,68.62,141.32,33.66,32.74,138.68,103.55,162.42,⋯,159.77,121.53,183.29,175.34,156.22,164.38,120.19,132.60,175.81,191.01
ENSMUSG00000000003_Pbsn,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000000028_Cdc45,5.29,4.60,5.39,1.43,3.90,1.90,2.33,3.65,8.25,5.82,⋯,26.31,28.41,22.41,23.37,21.64,14.82,31.30,12.53,36.95,31.72
ENSMUSG00000000031_H19,9.75,12.62,5.68,2.02,1.38,1.72,2.05,5.48,5.61,35.36,⋯,75.14,161.20,58.76,66.95,42.86,35.30,71.70,34.19,25.50,45.81
ENSMUSG00000000037_Scml2,1.46,2.56,1.23,0.01,0.63,0.39,0.20,0.82,1.77,1.54,⋯,2.56,2.72,2.86,2.26,4.80,4.08,1.39,1.37,3.17,0.53


In [433]:
## subset to good reps
good_reps=read.table('/mnt/lab_data2/annashch/nobel_lab_projects/age_V2/RNA_seq/good_reps_counts_batches/good_reps.txt',header=FALSE,stringsAsFactors=FALSE)

In [434]:
good_reps$V1

[1] "d0_A_Rep1"      "d0_A_Rep2"      "d0_A_Rep4"      "d0_Y_Rep2"     
 [5] "d0_Y_Rep3"      "d0_Y_Rep4"      "d0_Y_Rep5"      "d0_Y_Rep6"     
 [9] "d0_Y_Pax7_Rep1" "d1_A_Rep1"      "d1_A_Rep2"      "d1_Y_Rep1"     
[13] "d1_Y_Rep2"      "d1_Y_Rep4"      "d3_A_Rep1"      "d3_A_Rep2"     
[17] "d3_A_Rep3"      "d3_A_Rep4"      "d3_Y_Rep1"      "d3_Y_Rep2"     
[21] "d3_Y_Rep3"      "d5_A_Rep1"      "d5_A_Rep2"      "d5_Y_Rep1"     
[25] "d5_Y_Rep2"      "d5_Y_Rep3"      "d7_A_Rep1"      "d7_A_Rep2"     
[29] "d7_A_Rep4"      "d7_Y_Rep1"      "d7_Y_Rep2"      "d7_Y_Rep3"     
[33] "d21_A_Rep1"     "d21_A_Rep2"     "d21_Y_Rep2"     "d21_Y_Rep3"

In [435]:
#let's exclude pax7 sample from analysis 
good_reps=good_reps[!good_reps$V1=="d0_Y_Pax7_Rep1",]


In [436]:
good_reps

[1] "d0_A_Rep1"  "d0_A_Rep2"  "d0_A_Rep4"  "d0_Y_Rep2"  "d0_Y_Rep3" 
 [6] "d0_Y_Rep4"  "d0_Y_Rep5"  "d0_Y_Rep6"  "d1_A_Rep1"  "d1_A_Rep2" 
[11] "d1_Y_Rep1"  "d1_Y_Rep2"  "d1_Y_Rep4"  "d3_A_Rep1"  "d3_A_Rep2" 
[16] "d3_A_Rep3"  "d3_A_Rep4"  "d3_Y_Rep1"  "d3_Y_Rep2"  "d3_Y_Rep3" 
[21] "d5_A_Rep1"  "d5_A_Rep2"  "d5_Y_Rep1"  "d5_Y_Rep2"  "d5_Y_Rep3" 
[26] "d7_A_Rep1"  "d7_A_Rep2"  "d7_A_Rep4"  "d7_Y_Rep1"  "d7_Y_Rep2" 
[31] "d7_Y_Rep3"  "d21_A_Rep1" "d21_A_Rep2" "d21_Y_Rep2" "d21_Y_Rep3"

In [437]:
data_good_reps=subset(data,select=good_reps)

In [438]:
write.table(data_good_reps,file="age.rna.counts.goodreps.tpm",sep='\t')

In [439]:
colnames(data)

[1] "d21_A_Rep1"     "d21_A_Rep2"     "d21_Y_Rep1"     "d21_Y_Rep2"    
 [5] "d21_Y_Rep3"     "d0_A_Rep1"      "d0_A_Rep2"      "d0_A_Rep3"     
 [9] "d0_A_Rep4"      "d0_Y_Pax7_Rep1" "d0_Y_Pax7_Rep2" "d0_Y_Pax7_Rep3"
[13] "d0_Y_Rep1"      "d0_Y_Rep2"      "d0_Y_Rep3"      "d0_Y_Rep4"     
[17] "d0_Y_Rep5"      "d0_Y_Rep6"      "d1_A_Rep1"      "d1_A_Rep2"     
[21] "d1_Y_Rep1"      "d1_Y_Rep2"      "d1_Y_Rep3"      "d1_Y_Rep4"     
[25] "d3_A_Rep1"      "d3_A_Rep2"      "d3_A_Rep3"      "d3_A_Rep4"     
[29] "d3_Y_Rep1"      "d3_Y_Rep2"      "d3_Y_Rep3"      "d3_Y_Rep4"     
[33] "d3_Y_Rep5"      "d5_A_Rep1"      "d5_A_Rep2"      "d5_Y_Rep1"     
[37] "d5_Y_Rep2"      "d5_Y_Rep3"      "d7_A_Rep1"      "d7_A_Rep2"     
[41] "d7_A_Rep3"      "d7_A_Rep4"      "d7_Y_Rep1"      "d7_Y_Rep2"     
[45] "d7_Y_Rep3"      "d7_Y_Rep4"

In [440]:
colnames(data_good_reps)

[1] "d0_A_Rep1"  "d0_A_Rep2"  "d0_A_Rep4"  "d0_Y_Rep2"  "d0_Y_Rep3" 
 [6] "d0_Y_Rep4"  "d0_Y_Rep5"  "d0_Y_Rep6"  "d1_A_Rep1"  "d1_A_Rep2" 
[11] "d1_Y_Rep1"  "d1_Y_Rep2"  "d1_Y_Rep4"  "d3_A_Rep1"  "d3_A_Rep2" 
[16] "d3_A_Rep3"  "d3_A_Rep4"  "d3_Y_Rep1"  "d3_Y_Rep2"  "d3_Y_Rep3" 
[21] "d5_A_Rep1"  "d5_A_Rep2"  "d5_Y_Rep1"  "d5_Y_Rep2"  "d5_Y_Rep3" 
[26] "d7_A_Rep1"  "d7_A_Rep2"  "d7_A_Rep4"  "d7_Y_Rep1"  "d7_Y_Rep2" 
[31] "d7_Y_Rep3"  "d21_A_Rep1" "d21_A_Rep2" "d21_Y_Rep2" "d21_Y_Rep3"

In [441]:
head(data_good_reps)
print(ncol(data))
print(ncol(data_good_reps))


,d0_A_Rep1,d0_A_Rep2,d0_A_Rep4,d0_Y_Rep2,d0_Y_Rep3,d0_Y_Rep4,d0_Y_Rep5,d0_Y_Rep6,d1_A_Rep1,d1_A_Rep2,⋯,d7_A_Rep1,d7_A_Rep2,d7_A_Rep4,d7_Y_Rep1,d7_Y_Rep2,d7_Y_Rep3,d21_A_Rep1,d21_A_Rep2,d21_Y_Rep2,d21_Y_Rep3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSID_gene_GeneName,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000000001_Gnai3,33.66,32.74,103.55,157.71,63.06,102.83,231.55,174.64,149.95,155.51,⋯,183.29,175.34,164.38,120.19,132.60,175.81,153.01,153.94,68.62,141.32
ENSMUSG00000000003_Pbsn,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSMUSG00000000028_Cdc45,1.90,2.33,8.25,7.65,6.26,5.88,32.78,3.08,47.59,42.55,⋯,22.41,23.37,14.82,31.30,12.53,36.95,5.29,4.60,1.43,3.90
ENSMUSG00000000031_H19,1.72,2.05,5.61,0.66,4.91,2.56,1.71,0.00,4.21,5.91,⋯,58.76,66.95,35.30,71.70,34.19,25.50,9.75,12.62,2.02,1.38
ENSMUSG00000000037_Scml2,0.39,0.20,1.77,1.02,0.50,0.53,2.38,2.13,1.99,1.49,⋯,2.86,2.26,4.08,1.39,1.37,3.17,1.46,2.56,0.01,0.63


[1] 46
[1] 35


In [442]:
## batches 
batches=read.table("/mnt/lab_data2/annashch/nobel_lab_projects/age_V2/RNA_seq/merged_rsem/batches.tsv",header=TRUE,sep='\t')
##subset to good reps 
batches=batches[batches$Rep %in% good_reps,]
batches$Batch=factor(batches$Batch)
batches$Pax7=NULL
batches$Age=factor(batches$Age)
batches$Day=factor(batches$Day)
batches$File=NULL
print(nrow(batches))
row.names(batches)=batches$Rep
batches


[1] 35


,Day,Age,Batch,Rep
,<fct>,<fct>,<fct>,<fct>
d0_A_Rep1,0,A,R2200,d0_A_Rep1
d0_A_Rep2,0,A,R2081,d0_A_Rep2
d0_A_Rep4,0,A,R2830,d0_A_Rep4
d0_Y_Rep2,0,Y,R2081,d0_Y_Rep2
d0_Y_Rep3,0,Y,R2081,d0_Y_Rep3
d0_Y_Rep4,0,Y,R2081,d0_Y_Rep4
d0_Y_Rep5,0,Y,R2879,d0_Y_Rep5
d0_Y_Rep6,0,Y,R2879,d0_Y_Rep6
d1_A_Rep1,1,A,R2278,d1_A_Rep1


In [443]:
write.table(batches,file="batches.goodreps.txt",sep='\t')

In [444]:
#remove genes that are not expressed in the good reps
print(nrow(data_good_reps))
data_good_reps=data_good_reps[rowSums(data_good_reps)>0,]
print(nrow(data_good_reps))

[1] 49672
[1] 36597


In [445]:
#use asinh(tpm)
E=as.matrix(asinh(data_good_reps))

## Correlation 

In [446]:
spearman_cor=cor(E, method = "spearman")

In [447]:
pearson_cor=cor(E,method="pearson")

In [448]:
write.table(spearman_cor,file="rna.uncorrected.spearman_r.tsv",quote=FALSE)
write.table(pearson_cor,file="rna.uncorrected.pearson_r.tsv",quote=FALSE)

In [449]:
## plot correlation heatmaps 
library(gplots)
require(gtools)
require(RColorBrewer)
cols <- colorRampPalette(brewer.pal(10, "RdBu"))(256)

svg(filename="RNA_rep_spearman_cor_preSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(spearman_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(10,10),
          main="RNA-seq replicates spearman correlation -- pre-SVA")
dev.off() 


png 
  2

In [450]:
svg(filename="RNA_rep_pearson_cor_preSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(pearson_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(10,10),          
          main="RNA-seq replicates pearson correlation -- pre-SVA")

dev.off() 

png 
  2

## Run PCA 

In [451]:
data.pca=prcomp(t(E))

In [452]:
var_explained=as.character(round(100*data.pca$sdev^2/sum(data.pca$sdev^2),2))

In [453]:
dev.off()

null device 
          1

In [454]:
svg(filename="age_rna_uncorrected_scree.svg",
   height=3,
   width=4,
   pointsize=12)
   print(barplot(100*data.pca$sdev^2/sum(data.pca$sdev^2),las=2,ylab="% Variance Explained",xlab="Principal Component",ylim=c(0,40), xlim=c(0,10)))
dev.off() 

      [,1]
 [1,]  0.7
 [2,]  1.9
 [3,]  3.1
 [4,]  4.3
 [5,]  5.5
 [6,]  6.7
 [7,]  7.9
 [8,]  9.1
 [9,] 10.3
[10,] 11.5
[11,] 12.7
[12,] 13.9
[13,] 15.1
[14,] 16.3
[15,] 17.5
[16,] 18.7
[17,] 19.9
[18,] 21.1
[19,] 22.3
[20,] 23.5
[21,] 24.7
[22,] 25.9
[23,] 27.1
[24,] 28.3
[25,] 29.5
[26,] 30.7
[27,] 31.9
[28,] 33.1
[29,] 34.3
[30,] 35.5
[31,] 36.7
[32,] 37.9
[33,] 39.1
[34,] 40.3
[35,] 41.5


png 
  2

In [455]:
pca_df=data.frame(data.pca$x)
pca_df$Rep=row.names(pca_df)
pca_df=merge(pca_df,batches,by="Rep")
head(pca_df)
print(nrow(pca_df))

Rep,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,⋯,PC29,PC30,PC31,PC32,PC33,PC34,PC35,Day,Age,Batch
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>
d0_A_Rep1,-107.57587,-8.442197,43.306329,-4.198188,21.7547174,-9.271020,1.016094,-11.309333,27.191797,⋯,2.4432776,4.3046636,0.4504537,0.5663467,-0.5383441,-0.6447318489,1.289698e-13,0,A,R2200
d0_A_Rep2,-112.46382,-12.360600,49.431367,8.688269,-0.7322842,-7.615856,-2.226438,-2.273251,27.284278,⋯,-2.1544832,-4.6472175,-0.2846177,-0.7607864,0.9323633,0.5603740634,1.569052e-12,0,A,R2081
d0_A_Rep4,36.46299,71.931319,-8.921969,9.442522,-8.9199091,-5.734627,30.941896,-1.986982,25.457701,⋯,0.8953097,-0.5392288,-0.1745708,0.2006001,-0.2013357,0.0007423778,-2.410179e-13,0,A,R2830
d0_Y_Rep2,-50.00529,21.281126,16.025064,31.400858,-36.6845670,23.150849,-25.282812,6.947738,-43.187285,⋯,-1.5163690,-5.2724750,-1.0983706,-0.2215520,-1.2265246,-0.1017336283,7.834447e-13,0,Y,R2081
d0_Y_Rep3,-40.19371,8.529906,9.593668,28.004610,-35.1224441,-6.134494,4.860718,7.011797,10.153483,⋯,0.3927164,-9.0064118,-1.3648438,1.0384484,-1.9043885,-0.2067361234,-1.363657e-13,0,Y,R2081
d0_Y_Rep4,-54.84588,8.120985,-18.388118,16.989568,-45.3585576,9.811569,-3.462773,9.828005,-9.591366,⋯,3.1268924,18.1772421,3.6293175,-0.2917695,3.8249791,0.1862922388,-1.376165e-13,0,Y,R2081


[1] 35


In [456]:
svg(filename="age_rna_uncorrected_PC1_vs_PC2_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC2,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#e6ab02'))+
theme_bw() )
dev.off() 

png 
  2

In [457]:
svg(filename="age_rna_uncorrected_PC2_vs_PC3_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC2,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#e6ab02'))+
theme_bw())
dev.off() 

png 
  2

In [458]:
svg(filename="age_rna_uncorrected_PC1_vs_PC3_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#e6ab02'))+
theme_bw())
dev.off() 

png 
  2

## Running SVA correction 

In [459]:
head(batches)

,Day,Age,Batch,Rep
,<fct>,<fct>,<fct>,<fct>
d0_A_Rep1,0,A,R2200,d0_A_Rep1
d0_A_Rep2,0,A,R2081,d0_A_Rep2
d0_A_Rep4,0,A,R2830,d0_A_Rep4
d0_Y_Rep2,0,Y,R2081,d0_Y_Rep2
d0_Y_Rep3,0,Y,R2081,d0_Y_Rep3
d0_Y_Rep4,0,Y,R2081,d0_Y_Rep4


In [460]:
Grouping <- factor(paste0(batches$Day,".",batches$Age))
batches$Grouping=Grouping

In [461]:
mod0=model.matrix(~1,data=batches)
#mod1=model.matrix(~0+Day+Age,data=batches)
mod1=model.matrix(~0+Grouping,data=batches)
#mod1=model.matrix(~0+Age+Day,data=batches)

In [462]:
batches

,Day,Age,Batch,Rep,Grouping
,<fct>,<fct>,<fct>,<fct>,<fct>
d0_A_Rep1,0,A,R2200,d0_A_Rep1,0.A
d0_A_Rep2,0,A,R2081,d0_A_Rep2,0.A
d0_A_Rep4,0,A,R2830,d0_A_Rep4,0.A
d0_Y_Rep2,0,Y,R2081,d0_Y_Rep2,0.Y
d0_Y_Rep3,0,Y,R2081,d0_Y_Rep3,0.Y
d0_Y_Rep4,0,Y,R2081,d0_Y_Rep4,0.Y
d0_Y_Rep5,0,Y,R2879,d0_Y_Rep5,0.Y
d0_Y_Rep6,0,Y,R2879,d0_Y_Rep6,0.Y
d1_A_Rep1,1,A,R2278,d1_A_Rep1,1.A


In [463]:
sva.obj=sva(E,mod1,mod0)

Number of significant surrogate variables is:  5 
Iteration (out of 5 ):1  2  3  4  5  

In [464]:
summary(lm(sva.obj$sv ~ batches$Grouping))


Response Y1 :

Call:
lm(formula = Y1 ~ batches$Grouping)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.38828 -0.04963  0.00153  0.06276  0.24209 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)  
(Intercept)           0.11606    0.09982   1.163   0.2569  
batches$Grouping0.Y  -0.07606    0.12626  -0.602   0.5528  
batches$Grouping1.A  -0.12412    0.15783  -0.786   0.4396  
batches$Grouping1.Y  -0.18117    0.14117  -1.283   0.2121  
batches$Grouping21.A -0.31808    0.15783  -2.015   0.0557 .
batches$Grouping21.Y -0.03215    0.15783  -0.204   0.8404  
batches$Grouping3.A  -0.13745    0.13205  -1.041   0.3087  
batches$Grouping3.Y  -0.01837    0.14117  -0.130   0.8976  
batches$Grouping5.A  -0.01957    0.15783  -0.124   0.9024  
batches$Grouping5.Y  -0.04824    0.14117  -0.342   0.7357  
batches$Grouping7.A  -0.27412    0.14117  -1.942   0.0645 .
batches$Grouping7.Y  -0.19286    0.14117  -1.366   0.1851  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 

In [465]:
sur_var=data.frame(sva.obj$sv)
sur_var=subset(sur_var,select=c("X1","X4","X5"))

In [466]:
head(sur_var)

X1,X4,X5
<dbl>,<dbl>,<dbl>
0.1716799,-0.32124644,0.1655480
0.3581569,-0.18743028,0.1693453
-0.1816449,0.14625943,-0.2308775
0.2356109,0.00117383,-0.1461029
0.2740739,0.05809736,-0.1108559
0.2752381,0.03393413,-0.1307943


In [467]:
full.design.sv=cbind(mod1,sur_var)


In [468]:
#save the full design so we don't have to run SVA next time 
write.table(full.design.sv,file="full.design.sv.txt",quote=FALSE,sep='\t')


In [469]:
#full.design.sv=read.table("full.design.sv.txt",header=TRUE,sep='\t')


### Perform pca on the "cleaned" matrix with surrogate variable contributions removed


In [470]:
nrow(batches)

[1] 35

In [471]:

#cleaned_E=removeBatchEffect(E,covariates=sur_var,design=model.matrix(~0+Day+Age,data=batches))
cleaned_E=removeBatchEffect(E,covariates=sur_var,design=model.matrix(~0+Grouping,data=batches))
#cleaned_E=removeBatchEffect(E,covariates=sur_var,design=model.matrix(~0+Age+Day,data=batches))

In [472]:
write.table(cleaned_E,file="RNA_asinh_tpm_minus_sva_contribs.txt",sep='\t')

In [475]:
write.table(sinh(cleaned_E),file="RNA_transformed_tpm_minus_sva_contribs.txt",sep='\t')

In [362]:
## Run replicate correlation on SVA corrected dataset 
spearman_cor=cor(cleaned_E, method = "spearman")
pearson_cor=cor(cleaned_E,method="pearson")

In [363]:
write.table(spearman_cor,file="rna.corrected.spearman_r.tsv",quote=FALSE)
write.table(pearson_cor,file="rna.corrected.pearson_r.tsv",quote=FALSE)

In [364]:

svg(filename="RNA_rep_spearman_cor_postSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(spearman_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(12,12),          
          main="RNA-seq replicates spearman correlation -- post-SVA ")
dev.off() 



png 
  2

In [365]:
svg(filename="RNA_rep_pearson_cor_postSVA.svg",
   height=8,
   width=8,
   pointsize=12)

cur_h=heatmap.2(as.matrix(pearson_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(12,12),          
          main="RNA-seq replicates pearson correlation -- post SVA")
dev.off() 


png 
  2

In [366]:
data.pca=prcomp(t(cleaned_E))

In [367]:
var_explained=as.character(round(100*data.pca$sdev^2/sum(data.pca$sdev^2),2))


In [368]:
svg(filename="age_rna_corrected_scree.svg",
   height=3,
   width=4,
   pointsize=12)
print(barplot(100*data.pca$sdev^2/sum(data.pca$sdev^2),las=2,ylab="% Variance Explained",xlab="Principal Component",ylim=c(0,100), xlim=c(0,10)))
dev.off()

      [,1]
 [1,]  0.7
 [2,]  1.9
 [3,]  3.1
 [4,]  4.3
 [5,]  5.5
 [6,]  6.7
 [7,]  7.9
 [8,]  9.1
 [9,] 10.3
[10,] 11.5
[11,] 12.7
[12,] 13.9
[13,] 15.1
[14,] 16.3
[15,] 17.5
[16,] 18.7
[17,] 19.9
[18,] 21.1
[19,] 22.3
[20,] 23.5
[21,] 24.7
[22,] 25.9
[23,] 27.1
[24,] 28.3
[25,] 29.5
[26,] 30.7
[27,] 31.9
[28,] 33.1
[29,] 34.3
[30,] 35.5
[31,] 36.7
[32,] 37.9
[33,] 39.1
[34,] 40.3
[35,] 41.5


png 
  2

In [369]:
var_explained[0:10]


[1] "39.2"  "10.84" "7.8"   "6.45"  "5.68"  "3.88"  "3.3"   "2.9"   "2.77" 
[10] "2"

In [370]:
pca_df=data.frame(data.pca$x)
pca_df=cbind(pca_df,batches)
pca_df$Day=factor(pca_df$Day)

In [371]:
dev.off()

null device 
          1

In [385]:
svg(filename="age_rna_corrected_PC1_vs_PC2.svg",
   height=5,
   width=6,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC2,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(size=3,color='black',alpha=0.5)+
scale_shape(name="Age")+
scale_color_manual(name="Day After Injury", values=c('#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02'))+
xlab("PC1: 39.2%")+
ylab("PC2: 10.8%")+
theme_bw())
dev.off()

png 
  2

In [386]:
svg(filename="age_rna_corrected_PC2_vs_PC3.svg",
   height=5,
   width=6,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC2,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(size=3,color='black',alpha=0.5)+
scale_shape(name="Age")+
scale_color_manual(name="Day After Injury", values=c('#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02'))+
xlab("PC2: 10.8%")+
ylab("PC3: 7.8%")+
theme_bw())
dev.off()

png 
  2

In [387]:
svg(filename="age_rna_corrected_PC1_vs_PC3.svg",
   height=5,
   width=6,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(size=3,color='black',alpha=0.5)+
scale_shape(name="Age")+
scale_color_manual(name="Day After Injury", values=c('#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02'))+
xlab("PC1: 39.2%")+
ylab("PC3: 7.8%")+
theme_bw())
dev.off()

png 
  2

In [388]:
dev.off() 


null device 
          1

### fit model with limma 

In [389]:
#fit <- lmFit(E,full.design.sv)
fit = lmFit(cleaned_E,model.matrix(~0+Grouping,data=batches))

In [390]:
head(cleaned_E)

,d0_A_Rep1,d0_A_Rep2,d0_A_Rep4,d0_Y_Rep2,d0_Y_Rep3,d0_Y_Rep4,d0_Y_Rep5,d0_Y_Rep6,d1_A_Rep1,d1_A_Rep2,⋯,d7_A_Rep1,d7_A_Rep2,d7_A_Rep4,d7_Y_Rep1,d7_Y_Rep2,d7_Y_Rep3,d21_A_Rep1,d21_A_Rep2,d21_Y_Rep2,d21_Y_Rep3
ENSMUSG00000000001_Gnai3,4.6226989,4.8904272,4.9128842,6.10524960,5.2562732,5.7446914,5.58810647,5.6349763,5.64159999,5.681974,⋯,5.64285633,5.61001265,5.3673478,5.44731841,5.410556,5.6255309,5.38675977,5.3074516,5.1390564,5.8847222
ENSMUSG00000000028_Cdc45,1.5874087,2.0558406,2.4606588,2.87174703,2.7776097,2.6871836,3.38103177,1.8574525,4.50816979,4.378520,⋯,3.63541320,3.68763072,3.2131716,4.11214858,2.865086,4.5105968,2.01317494,1.8136211,1.2431797,2.0869658
ENSMUSG00000000031_H19,1.6986316,1.6636379,2.0946980,0.31397291,1.9719771,1.3389436,1.69360610,0.8123971,2.00201252,2.272811,⋯,4.65807531,4.77649015,4.2075902,5.00874749,3.638468,4.3909659,3.03748962,3.1223516,1.7417228,1.4367049
ENSMUSG00000000037_Scml2,0.9053910,1.0152058,0.9135097,1.39671241,1.0181678,1.0659666,1.26195193,1.0706260,1.38028081,1.154077,⋯,1.46888214,1.26208437,1.5343184,1.08983067,1.141233,1.2900676,0.87173691,1.2834680,0.2896408,0.9651325
ENSMUSG00000000049_Apoh,0.8556486,0.7585839,0.5228021,0.01492557,0.1172259,0.2602884,0.07322952,0.2581641,0.08951951,0.976122,⋯,0.08126926,0.08154572,0.5608123,0.09655915,0.213835,-0.1116508,0.07097623,0.1327868,-0.1109352,-0.1101817
ENSMUSG00000000056_Narf,2.5369584,2.3142511,2.6969144,3.13168890,3.1391551,2.9636860,2.89611748,3.3920972,1.96421992,2.433974,⋯,3.07044183,2.91165499,2.3348080,2.38754103,3.039408,2.8229758,3.14819519,3.2342959,2.7298025,3.0796836


In [391]:
colnames(fit)


[1] "Grouping0.A"  "Grouping0.Y"  "Grouping1.A"  "Grouping1.Y"  "Grouping21.A"
 [6] "Grouping21.Y" "Grouping3.A"  "Grouping3.Y"  "Grouping5.A"  "Grouping5.Y" 
[11] "Grouping7.A"  "Grouping7.Y"

In [392]:
colnames(fit$coefficients)

[1] "Grouping0.A"  "Grouping0.Y"  "Grouping1.A"  "Grouping1.Y"  "Grouping21.A"
 [6] "Grouping21.Y" "Grouping3.A"  "Grouping3.Y"  "Grouping5.A"  "Grouping5.Y" 
[11] "Grouping7.A"  "Grouping7.Y"

###  Contrasts 

In [393]:
#create contrasts of interest 
cont.matrix=makeContrasts(
    d0_Y_vs_A="Grouping0.Y - Grouping0.A",
    d1_Y_vs_A="Grouping1.Y - Grouping1.A",
    d3_Y_vs_A="Grouping3.Y - Grouping3.A",
    d5_Y_vs_A="Grouping5.Y - Grouping5.A",
    d7_Y_vs_A="Grouping7.Y - Grouping7.A",
    d21_Y_vs_A="Grouping21.Y - Grouping21.A",
    d1_Y_vs_d0_Y="Grouping1.Y - Grouping0.Y",
    d1_A_vs_d0_A="Grouping1.A - Grouping0.A",
    d3_Y_vs_d0_Y="Grouping3.Y - Grouping0.Y",
    d3_A_vs_d0_A="Grouping3.A - Grouping0.A",
    d5_Y_vs_d0_Y="Grouping5.Y - Grouping0.Y",
    d5_A_vs_d0_A="Grouping5.A - Grouping0.A",
    d7_Y_vs_d0_Y="Grouping7.Y - Grouping0.Y",
    d7_A_vs_d0_A="Grouping7.A - Grouping0.A",
    d21_Y_vs_d0_Y="Grouping21.Y - Grouping0.Y",
    d21_A_vs_d0_A="Grouping21.A - Grouping0.A",
      
    levels=model.matrix(~0+Grouping))


In [399]:
pval_thresh=0.01
lfc_thresh=1

In [400]:
fit2=contrasts.fit(fit,cont.matrix)
e=eBayes(fit2)
comparisons=colnames(cont.matrix)

In [401]:
comparisons

[1] "d0_Y_vs_A"     "d1_Y_vs_A"     "d3_Y_vs_A"     "d5_Y_vs_A"    
 [5] "d7_Y_vs_A"     "d21_Y_vs_A"    "d1_Y_vs_d0_Y"  "d1_A_vs_d0_A" 
 [9] "d3_Y_vs_d0_Y"  "d3_A_vs_d0_A"  "d5_Y_vs_d0_Y"  "d5_A_vs_d0_A" 
[13] "d7_Y_vs_d0_Y"  "d7_A_vs_d0_A"  "d21_Y_vs_d0_Y" "d21_A_vs_d0_A"

In [402]:
for(i in seq(1,length(comparisons)))
{
  tab<-topTable(e, number=nrow(e),coef=i,lfc=lfc_thresh, p.value = pval_thresh)
  up=sum(tab$logFC>0)
  down=sum(tab$logFC<0)
  sig=nrow(tab)
  curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')
  print(curtitle)
  vals=topTable(e,number=nrow(e),coef=i)
  vals$pscaled=-1*log10(vals$adj.P.Val)
  vals$sig=vals$adj.P.Val<pval_thresh & abs(vals$logFC)>lfc_thresh 
  png(paste("volcano_diff",comparisons[i],".png",sep=""))
  print(ggplot(data=vals,
               aes(y=vals$pscaled,x=vals$logFC,color=vals$sig))+
               geom_point(alpha=0.1)+
               xlab("log2(FC)")+
               ylab("-log10(pval)")+
               ggtitle(curtitle)+
               theme_bw()+
               scale_color_manual(values=c("#000000","#FF0000")))
  dev.off() 
  write.table(tab,file=paste("diff_",comparisons[i],".tsv",sep=""),quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}
    

[1] "d0_Y_vs_A \n sig: 552 \n up: 307 \n down: 245 \n"
[1] "d1_Y_vs_A \n sig: 1450 \n up: 646 \n down: 804 \n"
[1] "d3_Y_vs_A \n sig: 63 \n up: 53 \n down: 10 \n"
[1] "d5_Y_vs_A \n sig: 3038 \n up: 1810 \n down: 1228 \n"
[1] "d7_Y_vs_A \n sig: 109 \n up: 22 \n down: 87 \n"
[1] "d21_Y_vs_A \n sig: 968 \n up: 204 \n down: 764 \n"
[1] "d1_Y_vs_d0_Y \n sig: 839 \n up: 526 \n down: 313 \n"
[1] "d1_A_vs_d0_A \n sig: 1892 \n up: 1235 \n down: 657 \n"
[1] "d3_Y_vs_d0_Y \n sig: 4782 \n up: 2130 \n down: 2652 \n"
[1] "d3_A_vs_d0_A \n sig: 4011 \n up: 2196 \n down: 1815 \n"
[1] "d5_Y_vs_d0_Y \n sig: 2331 \n up: 1144 \n down: 1187 \n"
[1] "d5_A_vs_d0_A \n sig: 2282 \n up: 1316 \n down: 966 \n"
[1] "d7_Y_vs_d0_Y \n sig: 1529 \n up: 644 \n down: 885 \n"
[1] "d7_A_vs_d0_A \n sig: 1421 \n up: 865 \n down: 556 \n"
[1] "d21_Y_vs_d0_Y \n sig: 1187 \n up: 422 \n down: 765 \n"
[1] "d21_A_vs_d0_A \n sig: 475 \n up: 314 \n down: 161 \n"


In [404]:
cleaned_E_df=as.data.frame(cleaned_E)
head(cleaned_E_df)

,d0_A_Rep1,d0_A_Rep2,d0_A_Rep4,d0_Y_Rep2,d0_Y_Rep3,d0_Y_Rep4,d0_Y_Rep5,d0_Y_Rep6,d1_A_Rep1,d1_A_Rep2,⋯,d7_A_Rep1,d7_A_Rep2,d7_A_Rep4,d7_Y_Rep1,d7_Y_Rep2,d7_Y_Rep3,d21_A_Rep1,d21_A_Rep2,d21_Y_Rep2,d21_Y_Rep3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000000001_Gnai3,4.6226989,4.8904272,4.9128842,6.10524960,5.2562732,5.7446914,5.58810647,5.6349763,5.64159999,5.681974,⋯,5.64285633,5.61001265,5.3673478,5.44731841,5.410556,5.6255309,5.38675977,5.3074516,5.1390564,5.8847222
ENSMUSG00000000028_Cdc45,1.5874087,2.0558406,2.4606588,2.87174703,2.7776097,2.6871836,3.38103177,1.8574525,4.50816979,4.378520,⋯,3.63541320,3.68763072,3.2131716,4.11214858,2.865086,4.5105968,2.01317494,1.8136211,1.2431797,2.0869658
ENSMUSG00000000031_H19,1.6986316,1.6636379,2.0946980,0.31397291,1.9719771,1.3389436,1.69360610,0.8123971,2.00201252,2.272811,⋯,4.65807531,4.77649015,4.2075902,5.00874749,3.638468,4.3909659,3.03748962,3.1223516,1.7417228,1.4367049
ENSMUSG00000000037_Scml2,0.9053910,1.0152058,0.9135097,1.39671241,1.0181678,1.0659666,1.26195193,1.0706260,1.38028081,1.154077,⋯,1.46888214,1.26208437,1.5343184,1.08983067,1.141233,1.2900676,0.87173691,1.2834680,0.2896408,0.9651325
ENSMUSG00000000049_Apoh,0.8556486,0.7585839,0.5228021,0.01492557,0.1172259,0.2602884,0.07322952,0.2581641,0.08951951,0.976122,⋯,0.08126926,0.08154572,0.5608123,0.09655915,0.213835,-0.1116508,0.07097623,0.1327868,-0.1109352,-0.1101817
ENSMUSG00000000056_Narf,2.5369584,2.3142511,2.6969144,3.13168890,3.1391551,2.9636860,2.89611748,3.3920972,1.96421992,2.433974,⋯,3.07044183,2.91165499,2.3348080,2.38754103,3.039408,2.8229758,3.14819519,3.2342959,2.7298025,3.0796836


## Ddit3

In [405]:
ddit3_subset=sinh(cleaned_E_df[rownames(cleaned_E_df)=="ENSMUSG00000025408_Ddit3",])

In [406]:
write.table(ddit3_subset,file="DDIT3_tpm_svacorrected.txt")